In [12]:
import numpy as np
import torch
import torch.nn as nn
import time 
import logging
import torch.optim as optim
import os
from scipy.stats import multivariate_normal as normal
import torch.nn.functional as F
from torch.nn import Parameter
import matplotlib.pyplot as plt
import torchvision

In [3]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [28]:
%matplotlib inline
torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [29]:
from torchvision import transforms

In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

data_type=torch.float32
MOMENTUM = 0.99
EPSILON = 1e-6

Using cuda device


# Handling the data

In [31]:
from torchvision import datasets 
batch_size_train=cfg.batch_size
batch_size_test=cfg.batch_size

In [32]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [33]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

We have stored both the training and the validation datasets

Defining the dataloader

## Defining the configuration

In [56]:
class Config(object):
    batch_size = 200
    
    totalT=2.0;
    
    n_layer=Ntime=4; 
    
    sqrt_deltaT=np.sqrt(totalT/Ntime); 

    logging_frequency = 100
    verbose = True
   
    input_chanel=1
    output_chanel_pj1=32
    output_chanel_pj2=16 
    
    unflatten_shape=output_chanel_pj2*7*7
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

In [57]:
cfg=get_config('Config')

# Constructing a dense net

## Building the building block

In [66]:
class ProjBlock(nn.Module):
    def __init__(self,input_chanel,output_chanel):
        super(ProjBlock,self).__init__()
        self.input_chanel=input_chanel
        self.output_chanel=output_chanel
        
        self.conv1=nn.Conv2d(input_chanel,output_chanel,kernel_size=3,padding=1) 
        self.act1=nn.Tanh()
        self.pool1=nn.MaxPool2d(2)
        
      #  self.conv2=nn.Conv2d(2*output_chanel,output_chanel,kernel_size=3,padding=1) 
      #  self.act1=nn.Tanh()
      #  self.pool1=nn.MaxPool2d(2)
    
    def forward(self,x):
        out = self.pool1(self.act1(self.conv1(x)))
      #  out = self.pool2(self.act2(self.conv2(x)))
        return out

In [67]:
class BasicBlock(nn.Module):
    def __init__(self,num_chanel):
        super(BasicBlock,self).__init__()
        self.input_chanel=num_chanel
        self.output_chanel=num_chanel
        
        self.conv=nn.Conv2d(self.input_chanel,self.output_chanel,kernel_size=3,padding=1)
        self.act=nn.Tanh()
        ## there should not be any MaxPooling layer in the inbetween set
        
    def forward(self,x):
        out=self.act(self.conv(x))
        return out

In [68]:
# One is responsible for figuring out the unflatten shape
class FullyConnected(nn.Module):
    def __init__(self,unflatten_shape): 
        super(FullyConnected,self).__init__()
        self.unflatten_shape=unflatten_shape
        self.fc1=nn.Linear(unflatten_shape,32)
        self.ac1=nn.Tanh()
        self.fc2=nn.Linear(32,10) 
        # Let's only tell the airplane from a bird
    
    def forward(self,x):
        inputx=x.view(-1, self.unflatten_shape)
        out=self.fc2(self.ac1(self.fc1(inputx)))
        return out

## Stacking up the blocks

In [69]:
loss_fn=nn.CrossEntropyLoss()
class ForwardModel(nn.Module):
    def __init__(self,config):
        super(ForwardModel,self).__init__()
        
        self.config=config
        self.batch_size=self.config.batch_size
        self.Ntime=self.config.Ntime
        self.sqrt_deltaT=self.config.sqrt_deltaT;
        self.n_layer=self.config.n_layer
        self.delta=self.config.totalT/self.Ntime;
        
        self.mList=nn.ModuleList([ProjBlock(self.config.input_chanel,self.config.output_chanel_pj1),
                                  ProjBlock(self.config.output_chanel_pj1,self.config.output_chanel_pj2),
                                  BasicBlock(self.config.output_chanel_pj2),
                                  BasicBlock(self.config.output_chanel_pj2),
                                  BasicBlock(self.config.output_chanel_pj2),
                                  BasicBlock(self.config.output_chanel_pj2),
                                  FullyConnected(self.config.unflatten_shape)                              
        ])
        
        self.sigma=0.05
        
    def forwardX(self,x):# here x is the batch collection of images
        
        # Constructing the noises
        # The number 8 is determined from the number of max-pooling size, kernels & paddings etc. 
        xMat=[]
        wMat=torch.FloatTensor(normal.rvs(size=[self.batch_size,
                                     self.config.output_chanel_pj2,7,7,
                                     self.Ntime]) * self.sqrt_deltaT).to(device)
        x0=torch.clone(x).to(device); 
        xMat.append(x0); 
        
        x_pj1=self.mList[0](x0); 
        xMat.append(x_pj1.to(device)); 
        x_input=self.mList[1](x_pj1)
        xMat.append(x_input.to(device));
        
        for i in range(self.Ntime):
            # i + 2 because we already have two layers before
            xtemp=xMat[i+2]+self.mList[i+2](xMat[i+2])*self.delta #+self.sigma*wMat[:,:,:,:,i] 
            xMat.append(xtemp.to(device))
        
        x_terminal=self.mList[-1](xMat[-1])
        xMat.append(x_terminal.to(device))
        
        
        return xMat, wMat
        
        # The input of the target must be a tensor not a list
    def backwardYZ(self,xMat,wMat,target):
        yMat=[];  
        
        L=len(xMat)
        x_terminal=xMat[-1].to(device)
        
        loss_val=loss_fn(x_terminal,target.to(device))
        loss_val.to(device); 
        
        y_terminal=torch.autograd.grad(outputs=[loss_val], inputs=[x_terminal], grad_outputs=torch.ones_like(loss_val), allow_unused=True,
                                 retain_graph=True, create_graph=True)[0]
        #Here y_terminal has dim batch_size x output_size (2 x 2)
        yMat.append(y_terminal.to(device)); 
        xtemp=xMat[L-2].to(device) # 3 
        
        ## Finding Y[T-1]
        hami=torch.sum(y_terminal.detach()*self.mList[-1](xtemp),dim=1,keepdim=True) # keep dim=1 is correct
        hami=hami.view(-1,1);hami.to(device)

        hami_x=torch.autograd.grad(outputs=[hami], inputs=[xtemp], grad_outputs=torch.ones_like(hami),allow_unused=True,
                                 retain_graph=True, create_graph=True)[0]
    
        yMat.append(hami_x.to(device))
        
        for i in range(self.Ntime-1,-1,-1):
            X=xMat[i+2].to(device); 
            hami=torch.sum(yMat[-1].detach()*self.mList[i+2](X),dim=(1,2,3))
            hami=hami.view(-1,1); hami.to(device); 
            
            hami_x=torch.autograd.grad(outputs=[hami], inputs=[X], grad_outputs=torch.ones_like(hami),allow_unused=True,
                                 retain_graph=True, create_graph=True)[0]
            ytemp=yMat[-1]+hami_x*self.delta

            yMat.append(ytemp.to(device))
       
    ### Second projection layer
        X=xMat[1].to(device); 
       # X.requires_grad
        hami=torch.sum(yMat[-1].detach()*self.mList[1](X),dim=(1,2,3))
        hami=hami.view(-1,1); hami.to(device); 
            
        ytemp=torch.autograd.grad(outputs=[hami], inputs=[X], grad_outputs=torch.ones_like(hami),allow_unused=True,
                                 retain_graph=True, create_graph=True)[0]
        yMat.append(ytemp.to(device))
        
        X=xMat[0].to(device); 
        X.requires_grad=True
        hami=torch.sum(yMat[-1].detach()*self.mList[0](X),dim=(1,2,3))
        hami=hami.view(-1,1); hami.to(device)
            
        ytemp=torch.autograd.grad(outputs=[hami], inputs=[X], grad_outputs=torch.ones_like(hami),allow_unused=True,
                                 retain_graph=True, create_graph=True)[0]
        yMat.append(ytemp.to(device))
        
        return yMat  #yMat the order is reversed 
    
    def HamCompute(self,xMat,yMat):
        totalham=0.0
        
        for i in range(self.Ntime+3):
            ham_temp=torch.sum(yMat[self.Ntime+2-i].detach()*self.mList[i](xMat[i].detach()) )  #inside the bracket =  +\small_value * self.batch_size *self.mList[i]*self.mList[i] (No, this doesn't contain batchsize)
            totalham+=ham_temp
        
        return totalham/self.batch_size/(self.Ntime+3)

In [70]:

def train_accuracy(train_loader):
  correct = 0
  total = 0

  with torch.no_grad():
      for imgs, labels in train_loader:
          outputs = net.forwardX(imgs)
          _, predicted = torch.max(outputs[0][-1], dim=1)
          total += labels.shape[0]
          correct += int((predicted == labels.to(device)).sum())
  res=correct/total

  return res

def test_accuracy(val_loader):
  correct = 0
  total = 0

  with torch.no_grad():
      for imgs, labels in val_loader:
          outputs =net.forwardX(imgs)
          _, predicted = torch.max(outputs[0][-1], dim=1)
          total += labels.shape[0]
          correct += int((predicted == labels.to(device)).sum())
  res=correct/total
  return res

In [71]:
n_epoch=20 

net=ForwardModel(cfg)
net.to(device)

optimizer=optim.Adam(net.parameters(), lr=1.5e-3)#it could be a bad idea to add weight decay
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1000,2500,4000], gamma=0.2) 

Loss_vec=[]
training_accuracy=[]
testing_accuracy=[]

for epoch in range(n_epoch):
    for imgs, labels in train_loader:
        

        xmat,wmat=net.forwardX(imgs); 
        ymat=net.backwardYZ(xmat,wmat.to(device),labels)
        loss_temp=net.HamCompute(xmat,ymat)
        loss_temp.to(device)
        
        optimizer.zero_grad();
        loss_temp.backward()
        optimizer.step()
        
    if epoch %2 ==0:
        loss_val=loss_fn(xmat[-1].to(device),labels.to(device))
       # ham_loss=net.HamCompute(xmat,ymat)
       # print(ham_loss.cpu().detach().numpy(), loss_val.cpu().detach().numpy())
        loss_val_np=loss_val.cpu().detach().numpy()
        print(epoch, loss_val_np)
        Loss_vec.append(loss_val_np)

    #if epoch %10 ==0: 
        test_temp=test_accuracy(test_loader)
        testing_accuracy.append(test_temp)
        print(test_temp)

        train_temp=train_accuracy(train_loader)
        training_accuracy.append(train_temp)
        print(test_temp)

0 0.09989919
0.9701
0.9701
2 0.05897376
0.9871
0.9871
4 0.015559707
0.9898
0.9898
6 0.06452933
0.9881
0.9881
8 0.01710263
0.989
0.989
10 0.003968168
0.99
0.99
12 0.014267882
0.9896
0.9896
14 0.0029396052
0.9885
0.9885
16 0.01590621
0.9865
0.9865
18 0.003950814
0.9893
0.9893


In [72]:
test_accuracy(test_loader)

0.9894

In [73]:
train_accuracy(train_loader)

0.9983

In [ ]:
testing_file = open("testing_accuracy"+str(cfg.batch_size)+"_003"+".txt", "w")
np.savetxt(testing_file, testing_accuracy)
testing_file.close()

In [ ]:
loss_file = open("loss"+str(cfg.batch_size)+"_003"+".txt", "w")
np.savetxt(loss_file, Loss_vec)
loss_file.close()

In [ ]:
training_file = open("training_accuracy"+str(cfg.batch_size)+"_003"+".txt", "w")
np.savetxt(training_file, training_accuracy)
training_file.close()

In [ ]:
for i in range(5, 125,5):
  print(i)

In [ ]:
test_accuracy(val_loader)

In [ ]:
plt.yscale('log')
plt.grid(True,which="both", linestyle='--')
plt.plot(range(5,125,5),Loss_vec)
plt.title('Train_loss', fontsize=10)
plt.savefig("train_loss.png", bbox_inches='tight')

In [ ]:
plt.yscale('log')
plt.grid(True,which="both", linestyle='--')
plt.plot(range(10,130,10),  testing_accuracy)
plt.title('testing_accuracy', fontsize=10)
plt.savefig("testing_accuracy.png", bbox_inches='tight')

In [ ]:
plt.yscale('log')
plt.grid(True,which="both", linestyle='--')
plt.plot(range(10,130,10),  training_accuracy)
plt.title('training_accuracy', fontsize=10)
plt.savefig("training_accuracy.png", bbox_inches='tight')

The following shows the test accuracy. It is only close to 90%

In [ ]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=100,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs =net.forwardX(imgs)
        _, predicted = torch.max(outputs[0][-1], dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels.to(device)).sum())
        
print("Accuracy: %f" % (correct / total))

In [ ]:
test_accuracy(val_loader)

We should test the case where the the data is attacked. 

In [ ]:
# Even when we use an average of the estimates, the results don't improve much
# at least for this simple case. 
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=100,
                                         shuffle=False)

The following is the training accuracy, and we see that it is very close to 1.00. Hence, the constructed neural network clearly has some overtraining issue. We need to use some standard regularization techinques. 

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=100,
                                           shuffle=False)

In [ ]:
train_accuracy(train_loader)